# cesm Rainfall Anomalies

In [1]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.timePeriod as tp

In [2]:
import xarray
import numpy
import cftime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
secondsPerDay = 60*60*24
secondsToTimeP = secondsPerDay*1000 #seconds per day, convert m to mm
#secondsToTimeP = secondsPerDay*_index.nDaysOfInterest["precAnom"]*1000/7 #seconds per month

# Carve out an area of interest, and calculate anomalies

In [6]:
#model='ACCESS-CM2'
for model in _model.scenarioMip:
    #Calculate a climatology
    #Based on the control run, calculate monthly anomalies
    print(model)
    xr = xarray.merge([
        fh.loadModelData(model[1], 'pr_Amon', 'piControl', model[2]),
        fh.loadModelData(model[1], 'tas_Amon', 'piControl', model[2])
    ], compat='override')
    monMeansDa=xr.groupby('time.month').mean(dim='time')
    monMeansDa=monMeansDa.where(
            (monMeansDa.lat>-50) & (monMeansDa.lat<0) & (monMeansDa.lon>100) & (monMeansDa.lon<170),
            drop=True
        )
    monMeansDa.to_netcdf('results/cmipMonthlyPrTs/monMeans'+model[1]+'.nc')

    
    
    monMeansDa=xarray.open_dataset(
        'results/cmipMonthlyPrTs/monMeans'+model[1]+'.nc')
    
    #grab area around Australia
    domainDa=xr.where(
        (xr.lat>-50) & (xr.lat<0) & (xr.lon>100) & (xr.lon<170),
        drop=True
    )
    
     #anom for piControl
    anomDa=domainDa.groupby('time.month')-monMeansDa
    anomDa['model']=model[1]

    anomDa.to_netcdf(
        'results/cmipMonthlyPrTs/'+model[1]+'PiControl.nc')
    anomDa.close()
    

    
    #calculate anomalies for all scenarios
    for experiment in _model.scenarioSet:
        
        try: 
            #load it
            xr = xarray.merge([
                fh.loadModelData(model[1], 'pr_Amon', experiment, model[3]),
                fh.loadModelData(model[1], 'tas_Amon', experiment, model[3])
            ], compat='override')

                    #grab area around Australia
            domainDa=xr.where(
                (xr.lat>-50) & (xr.lat<0) & (xr.lon>100) & (xr.lon<170),
                drop=True
            )

             #anom for piControl
            anomDa=domainDa.groupby('time.month')-monMeansDa
            anomDa['model']=model[1]
    
            anomDa.to_netcdf(
                'results/cmipMonthlyPrTs/'+model[1]+'_'+experiment+'.nc')
        except Exception as e:
            print(model[1] + experiment + " did not calculate")
            print(e)

['CSIRO-ARCCSS' 'ACCESS-CM2' 'r1i1p1f1' 'r1i1p1f1']
['CSIRO' 'ACCESS-ESM1-5' 'r1i1p1f1' 'r1i1p1f1']
['AWI' 'AWI-CM-1-1-MR' 'r1i1p1f1' 'r1i1p1f1']
['BCC' 'BCC-CSM2-MR' 'r1i1p1f1' 'r1i1p1f1']
['CAMS' 'CAMS-CSM1-0' 'r1i1p1f1' 'r1i1p1f1']
['CAS' 'CAS-ESM2-0' 'r1i1p1f1' 'r1i1p1f1']
['NCAR' 'CESM2' 'r1i1p1f1' 'r10i1p1f1']
['NCAR' 'CESM2-WACCM' 'r1i1p1f1' 'r1i1p1f1']
['THU' 'CIESM' 'r1i1p1f1' 'r1i1p1f1']
CIESMpr_Amonssp370r1i1p1f1 file not found on ESGF
CIESMssp370 did not calculate
Files (filter term: pr_Amon_.*?CIESM_ssp370_r1i1p1f1_.*?\.nc ) not found, possibly test name is wrong
['CMCC' 'CMCC-CM2-SR5' 'r1i1p1f1' 'r1i1p1f1']
['CMCC' 'CMCC-ESM2' 'r1i1p1f1' 'r1i1p1f1']
['CNRM-CERFACS' 'CNRM-CM6-1' 'r1i1p1f2' 'r1i1p1f2']
['CNRM-CERFACS' 'CNRM-ESM2-1' 'r1i1p1f2' 'r1i1p1f2']
['CCCma' 'CanESM5' 'r1i1p1f1' 'r1i1p1f1']
['CCCma' 'CanESM5-CanOE' 'r1i1p2f1' 'r1i1p2f1']
['EC-Earth-Consortium' 'EC-Earth3' 'r1i1p1f1' 'r1i1p1f1']
['EC-Earth-Consortium' 'EC-Earth3-CC' 'r1i1p1f1' 'r1i1p1f1']
EC-Earth3-CCpr

OSError: Files (filter term: tas_Amon_.*?GFDL-CM4_piControl_r1i1p1f1_.*?\.nc ) not found, possibly test name is wrong

# Time averaging for the anomalies

In [ ]:
for experiment in _model.experimentSet:
    monthlyAnomDa=xarray.concat([
        xarray.open_dataset(
            'results/cmipMonthlyPrTs/'+model+'_'+experiment+'.nc') for model in _model.scenarioMip],
    'model')
    

    monthlyAnomDa=monthlyAnomDa.assign({'model':_model.scenarioMip})
    warmSeasonAnomDa=xarray.DataArray()
    
    warmSeasonAnomDa['pr']=secondsToTimeP*tp.averageForTimePeriod(monthlyAnomDa.pr)
    warmSeasonAnomDa['ts']=tp.averageForTimePeriod(monthlyAnomDa.ts)
    warmSeasonAnomDa = warmSeasonAnomDa.assign_attrs([*warmSeasonAnomDa.attrs, ('units','mm/month'), ('timePeriod','Warm Season')])

    warmSeasonAnomDa.to_netcdf('results/cmip'+experiment.capitalize()+'PrecAnoms.nc')